In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.dates import  DateFormatter
# %matplotlib inline
from astropy.io import fits
import datetime
from astropy import units as u
from astropy.coordinates import SkyCoord, EarthLocation, AltAz
import time

In [2]:
from some_func import cart2spheric, spheric2cart, spheric_coordinates_rotate, hor2pix_keo, bright_relay_keo

In [3]:
# Координаты расположения KEO
lat_keo=55.9305361 # ШИРОТА
lon_keo=48.7444861 # ДОЛГОТА
h_keo=91.0
KEO_site=EarthLocation(lat=lat_keo*u.deg, lon=lon_keo*u.deg, height=h_keo*u.m)

# Ориентация KEO - 26 августа
az0_keo = 53.3287070263663878*np.pi/180;
alt0_keo = 88.4757875994250895*np.pi/180;
a_keo = (   1.7314835575767389,   -0.0028386665243216,   -0.0039615529590125)
b_keo = (   -0.2861392093442666,    0.0039634986844613,   -0.0028425834931615)
c_keo = ( 254.7431859558771237, -119.5787646993292981,  166.6502675414942019)
d_keo = ( 254.5346368979857630, -166.7359040180428167, -119.4151027599713473)

In [4]:
dark_filename='/mnt/Work_disk/Owncloud/0000_Two_points/DATA/02_KEO/140826/dark/keo_aug26_dark_median.fit'
dark=fits.getdata(dark_filename);
dark.astype('d')
BS6300_catalog_filename='Haritonov_6300BS_catalog.npz'

In [5]:
npzfile=np.load(BS6300_catalog_filename)
CAT_FIELDS=npzfile['CAT_FIELDS']
CAT_6300BS=npzfile['CAT_6300BS']
CAT_SPTYPE=npzfile['CAT_SPTYPE']
print(CAT_FIELDS)

['HARITONOV_ID' 'BS_ID' 'RA (icrs, deg)' 'DEC (icrs, deg)' 'MAG_V'
 '6300 Flux (erg/(sm^2 * s *  sm))']


In [7]:
# img_num_start=2971
img_num_start=2971
# img_num_end=2971
img_num_end=3464
img_num_list=[]
R_median_list=[]
R_median_sum=0.
time_axe=[]
ind=0
for img_num in range(img_num_start,img_num_end+1):
#     pic_save_name='/home/ashindin/temp/' + "{0:0>3}".format(img_num) + '.png'
    ind+=1
    pic_save_name='/home/ashindin/temp/keo_aug26/' + "{0:0>4}".format(ind) + '.jpg'
    fig=plt.figure(figsize=(9.5,4.5))
    ax = plt.subplot(121)
    ax.grid(True)
    ax.set_xlim((100, 1300))
    ax.set_ylim((0, 0.5))
    plt.ylabel('Calibration coef. [R/ADCu]')
    plt.xlabel('Star brightness per pixel [R]')
    ax2=plt.subplot(122)
    ax2.set_xlim((511,1))
    ax2.set_ylim((511,1))
        
    fit_filename='/mnt/Work_disk/Owncloud/0000_Two_points/DATA/02_KEO/140826/20140826_' + "{0:0>3}".format(img_num) + '.fit'
    
    fid_fit=fits.open(fit_filename);
    date_obs_str=fid_fit[0].header["DATE-OBS"]
    img=fid_fit[0].data;
    img=img.astype('d')
    fid_fit.close()
    
    img=img-dark
    plt.axes(ax2)
    plt.pcolormesh(img, cmap="gray", vmin=np.median(img)-300, vmax=np.median(img)+300)
    date_obs=datetime.datetime.strptime(date_obs_str,"%Y-%m-%dT%H:%M:%S")+datetime.timedelta(seconds=15)
    
    plt.title('fr'+"{0:0>4}".format(img_num) + ' ' + str(date_obs)[0:23])
    time_axe.append(date_obs)
    
    print("{0:0>3}".format(img_num) + " "+ str(date_obs))

    BS_coord=SkyCoord(CAT_6300BS[:,2], CAT_6300BS[:,3], frame='icrs', unit='deg');
    
    altaz=BS_coord.transform_to(AltAz(obstime=date_obs, location=KEO_site,temperature=15*u.deg_C,pressure=1013*u.hPa,
                                       relative_humidity=0.5,obswl=630.0*u.nm))
    BS_AzAlt=np.zeros((np.size(CAT_6300BS,0),2))
    BS_AzAlt[:,0]=altaz.az.degree
    BS_AzAlt[:,1]=altaz.alt.degree
#     print(BS_AzAlt[28,0],BS_AzAlt[28,1])

    BS_xy=np.zeros((np.size(CAT_6300BS,0),2))
    BS_xy[:,0], BS_xy[:,1] = hor2pix_keo (BS_AzAlt[:,0]*np.pi/180,BS_AzAlt[:,1]*np.pi/180, az0_keo, alt0_keo, c_keo, d_keo)

    # Catalog filtration
    filt_mask=np.zeros(np.size(CAT_6300BS,0),dtype=bool)
    for i in range(0,np.size(CAT_6300BS,0)):
        if BS_AzAlt[i,1]>=44.0 and BS_AzAlt[i,1]<=62.0:
            if BS_xy[i,0]>=1 and BS_xy[i,0]<=511:
                if BS_xy[i,1]>=1 and BS_xy[i,1]<=511:
                    filt_mask[i]=True
    BS_AzAlt_filt=BS_AzAlt[filt_mask,:]
    BS_xy_filt=BS_xy[filt_mask,:]
    CAT_6300BS_filt=CAT_6300BS[filt_mask,:]
    CAT_SPTYPE_filt=CAT_SPTYPE[filt_mask]
    if len(CAT_6300BS_filt)!=0:
        img_red_st_int=np.zeros(np.size(BS_xy_filt,0))
        img_red_st_num=np.zeros(np.size(BS_xy_filt,0))
        BS_adc_filt=np.zeros(np.size(BS_xy_filt,0))
        area_rad=2
        for j in range(np.size(BS_xy_filt,0)):    
            st_x=BS_xy_filt[j,0]
            st_y=BS_xy_filt[j,1]
            area=img[int(st_y)-area_rad:int(st_y)+area_rad+1, int(st_x)-area_rad:int(st_x)+area_rad+1]

            sum_temp=0.0
            num=0
            med=np.median(area)
            for i in range(len(area.flat)):
                if area.flat[i]>=1.3*med:
                    num+=1
                    sum_temp+=area.flat[i]-med
            img_red_st_int[j]=sum_temp
            img_red_st_num[j]=num
            if num>0:
                BS_adc_filt[j]=sum_temp/num

        BS_relay=np.zeros(np.size(BS_xy_filt,0))
        R_adc_coef=np.zeros(np.size(BS_xy_filt,0))
        for i in range(np.size(BS_xy_filt,0)):
             if img_red_st_num[i]>3:
                    BS_relay[i]=bright_relay_keo(CAT_6300BS_filt[i,5],img_red_st_num[i])
                    if BS_adc_filt[i]>0 and BS_relay[i]>100:
                        R_adc_coef[i]=BS_relay[i]/BS_adc_filt[i]

    #     print(R_adc_coef[R_adc_coef.nonzero()])
        R_median=np.median(R_adc_coef[R_adc_coef.nonzero()])
        R_median_sum+=R_median

        print(R_median)
        R_median_list.append(R_median)
        img_num_list.append(img_num)
        
        plt.axes(ax)
        plt.plot([0, 5000], [R_median, R_median], c='r', lw=2)
        plt.scatter(BS_relay,R_adc_coef,s=np.pi*(img_red_st_num)**2)
        plt.ylabel('Calibration coef. [R/ADCu]')
        plt.xlabel('Star brightness per pixel [R]')

        plt.title(R_median)

        plt.axes(ax2)
        plt.plot(BS_xy_filt[:,0],BS_xy_filt[:,1],color="r",marker=".", lw=0.,mec="r", mfc="r")
        
#         for ii in range(len(CAT_6300BS_filt)):
#             ax2.text(BS_xy_filt[ii,0]-10, BS_xy_filt[ii,1]-10, str(int(CAT_6300BS_filt[ii,0])), ha="center", va="center", size=12)
        
        
        # plt.show()
    plt.savefig(pic_save_name)
    plt.close()
print(' ')
print(R_median_sum/(img_num_end-img_num_start+1))

2971 2014-08-26 17:31:21
nan


/usr/local/lib/python3.5/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)
/usr/local/lib/python3.5/site-packages/numpy/core/_methods.py:70: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


2972 2014-08-26 17:31:52
nan
2973 2014-08-26 17:32:23
nan
2974 2014-08-26 17:32:54
nan
2975 2014-08-26 17:33:25
nan
2976 2014-08-26 17:33:56
nan
2977 2014-08-26 17:34:27
nan
2978 2014-08-26 17:34:58
0.669209906497
2979 2014-08-26 17:35:29
nan
2980 2014-08-26 17:36:00
nan
2981 2014-08-26 17:36:31
nan
2982 2014-08-26 17:37:01
nan
2983 2014-08-26 17:37:32
nan
2984 2014-08-26 17:38:03
nan
2985 2014-08-26 17:38:34
nan
2986 2014-08-26 17:39:05
nan
2987 2014-08-26 17:39:36
nan
2988 2014-08-26 17:40:07
nan
2989 2014-08-26 17:40:38
nan
2990 2014-08-26 17:41:09
nan
2991 2014-08-26 17:41:40
nan
2992 2014-08-26 17:42:11
nan
2993 2014-08-26 17:42:42
nan
2994 2014-08-26 17:43:13
nan
2995 2014-08-26 17:43:44
nan
2996 2014-08-26 17:44:15
nan
2997 2014-08-26 17:44:46
nan
2998 2014-08-26 17:45:17
nan
2999 2014-08-26 17:45:48
nan
3000 2014-08-26 17:46:19
nan
3001 2014-08-26 17:46:50
nan
3002 2014-08-26 17:47:21
nan
3003 2014-08-26 17:47:52
nan
3004 2014-08-26 17:48:22
nan
3005 2014-08-26 17:48:53
nan
300

In [9]:
dates = matplotlib.dates.date2num(time_axe)

In [12]:
fig=plt.figure(figsize=(9,8))
ax = plt.subplot(211)
# plt.scatter(img_num_list,R_median_list)
plt.plot_date(dates,R_median_list)
#ax.set_ylim((0,5))
ax.grid(True)
# plt.gcf().autofmt_xdate()
ax.xaxis.set_major_formatter( DateFormatter('%H:%M') )
plt.ylabel('Calibration coef. [R/ADCu]')

plt.title('KEO (26 aug 2014) calibration curve')

ax2=plt.subplot(212)
plt.plot_date(dates,R_median_list)
ax2.set_ylim((0.2,0.6))
ax2.grid(True)
# plt.gcf().autofmt_xdate()
ax2.xaxis.set_major_formatter( DateFormatter('%H:%M') )
plt.ylabel('Calibration coef. [R/ADCu]')

# plt.show()
plt.savefig('/home/ashindin/temp/keo_aug26.png')

In [6]:
img_num_start=3215
img_num_end=3351
# img_num_end=3215
R_median_sum=0.
for img_num in range(img_num_start,img_num_end+1):

    pic_save_name='/home/ashindin/temp/' + str(img_num) + '.png'

    fit_filename='/mnt/Work_disk/Owncloud/0000_Two_points/DATA/02_KEO/140826/20140826_' + str(img_num) + '.fit'

    fid_fit=fits.open(fit_filename);
    date_obs_str=fid_fit[0].header["DATE-OBS"]
    img=fid_fit[0].data;
    img=img.astype('d')
    fid_fit.close()
    
    img=img-dark
    date_obs=datetime.datetime.strptime(date_obs_str,"%Y-%m-%dT%H:%M:%S")+datetime.timedelta(seconds=15)
    print(date_obs)

    BS_coord=SkyCoord(CAT_6300BS[:,2], CAT_6300BS[:,3], frame='icrs', unit='deg');

    altaz=BS_coord.transform_to(AltAz(obstime=date_obs, location=KEO_site,temperature=15*u.deg_C,pressure=1013*u.hPa,
                                           relative_humidity=0.5,obswl=630.0*u.nm))
    BS_AzAlt=np.zeros((np.size(CAT_6300BS,0),2))
    BS_AzAlt[:,0]=altaz.az.degree
    BS_AzAlt[:,1]=altaz.alt.degree

#     print(BS_AzAlt[28,0],BS_AzAlt[28,1])

    BS_xy=np.zeros((np.size(CAT_6300BS,0),2))
    BS_xy[:,0], BS_xy[:,1] = hor2pix_keo (BS_AzAlt[:,0]*np.pi/180,BS_AzAlt[:,1]*np.pi/180, az0_keo, alt0_keo, c_keo, d_keo)

    # Catalog filtration
    filt_mask=np.zeros(np.size(CAT_6300BS,0),dtype=bool)
    for i in range(0,np.size(CAT_6300BS,0)):
        if BS_AzAlt[i,1]>=44.0 and BS_AzAlt[i,1]<=62.0:
            if BS_xy[i,0]>=1 and BS_xy[i,0]<=511:
                if BS_xy[i,1]>=1 and BS_xy[i,1]<=511:
                    filt_mask[i]=True
    BS_AzAlt_filt=BS_AzAlt[filt_mask,:]
    BS_xy_filt=BS_xy[filt_mask,:]
    CAT_6300BS_filt=CAT_6300BS[filt_mask,:]
    CAT_SPTYPE_filt=CAT_SPTYPE[filt_mask]

    img_red_st_int=np.zeros(np.size(BS_xy_filt,0))
    img_red_st_num=np.zeros(np.size(BS_xy_filt,0))
    BS_adc_filt=np.zeros(np.size(BS_xy_filt,0))
    area_rad=2
    for j in range(np.size(BS_xy_filt,0)):    
        st_x=BS_xy_filt[j,0]
        st_y=BS_xy_filt[j,1]
        area=img[int(st_y)-area_rad:int(st_y)+area_rad+1, int(st_x)-area_rad:int(st_x)+area_rad+1]

        sum_temp=0.0
        num=0
        med=np.median(area)
        for i in range(len(area.flat)):
            if area.flat[i]>=1.3*med:
                num+=1
                sum_temp+=area.flat[i]-med
        img_red_st_int[j]=sum_temp
        img_red_st_num[j]=num
        if num>0:
            BS_adc_filt[j]=sum_temp/num



    BS_relay=np.zeros(np.size(BS_xy_filt,0))
    R_adc_coef=np.zeros(np.size(BS_xy_filt,0))
    for i in range(np.size(BS_xy_filt,0)):
         if img_red_st_num[i]>3:
                BS_relay[i]=bright_relay_keo(CAT_6300BS_filt[i,5],img_red_st_num[i])
                if BS_adc_filt[i]>0 and BS_relay[i]>100:
                    R_adc_coef[i]=BS_relay[i]/BS_adc_filt[i]

#     print(R_adc_coef[R_adc_coef.nonzero()])
    R_median=np.median(R_adc_coef[R_adc_coef.nonzero()])
    R_median_sum+=R_median
    
    print(R_median)

    fig,ax = plt.subplots()
    plt.plot([0, 5000], [R_median, R_median], c='r', lw=2)
    plt.scatter(BS_relay,R_adc_coef,s=np.pi*(img_red_st_num)**2)
    plt.ylabel('Calibration coef. [R/ADCu]')
    plt.xlabel('Star brightness per pixel [R]')
    plt.title(date_obs, loc='left')
    plt.title(R_median, loc='right')
    ax.set_xlim((100, 1300))
    ax.set_ylim((0, 0.5))
    # plt.show()
    plt.savefig(pic_save_name)
    plt.close()
print(' ')
print(R_median_sum/(img_num_end-img_num_start+1))

2014-08-26 19:37:17
0.359219505386
2014-08-26 19:37:48
0.368755030283
2014-08-26 19:38:19
0.358338259937
2014-08-26 19:38:50
0.355663266245
2014-08-26 19:39:21
0.396776899703
2014-08-26 19:39:52
0.389739410131
2014-08-26 19:40:23
0.396653133263
2014-08-26 19:40:54
0.385220151109
2014-08-26 19:41:25
0.367876304992
2014-08-26 19:41:56
0.348650682225
2014-08-26 19:42:27
0.342946771517
2014-08-26 19:42:58
0.341985746902
2014-08-26 19:43:29
0.350350046507
2014-08-26 19:43:59
0.336710388489
2014-08-26 19:44:30
0.342456884512
2014-08-26 19:45:01
0.370508794264
2014-08-26 19:45:32
0.374120840941
2014-08-26 19:46:03
0.338333542531
2014-08-26 19:46:34
0.36686558441
2014-08-26 19:47:05
0.35437304434
2014-08-26 19:47:36
0.384913891604
2014-08-26 19:48:07
0.374424401587
2014-08-26 19:48:38
0.356339432151
2014-08-26 19:49:09
0.36188084062
2014-08-26 19:49:40
0.343195459484
2014-08-26 19:50:11
0.367675266493
2014-08-26 19:50:42
0.351549624216
2014-08-26 19:51:13
0.458607078688
2014-08-26 19:51:44
0.3